# LIBRARIES

In [ ]:
import pandas as pd
import numpy  as np

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import *
from pkg.neighborhood import *



from pkg.tabu_list import TabuList

from itertools import permutations
from random    import shuffle



# SEED PRODUCTION

In [ ]:
# seeds = np.random.randint(0,99999,SEEDS)
# print(seeds)

# Semillas adri
seeds = np.array([12345, 54321, 12221, 22211, 22212])
print(seeds)

# EVALUATION OBJECT

In [ ]:
evaluation = Evaluation()

# IMPLEMENATION OF ALGORITHIMS

## DATA DUMP

In [ ]:
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 
deltas_matrix   = deltas_df.to_numpy()

## GREEDY SEARCH

In [ ]:
initial_state = deltas_matrix[0,:]
greedy_sol, greedy_sol_cost = greedy(initial_state, evaluation)

In [ ]:
print(greedy_sol)
print(greedy_sol_cost)

## RANDOM SEARCH

In [ ]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    rs_sol, rs_sol_cost = random_search(evaluation = evaluation)

    print(rs_sol)
    print(rs_sol_cost)


## LOCAL SEARCH
first best neighbor

In [ ]:
slots_to_move = 1

for i in range(SEEDS):

    np.random.seed(seeds[i])

    ls_sol, ls_sol_cost = local_search(
        slots_to_move = slots_to_move,
        evaluation    = evaluation
    )
    
    print(ls_sol)
    print(ls_sol_cost)
  

## SIMULATED ANNEALING

In [ ]:
# parametros a ajustar:
slots = 1

# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20


# Esquema de enfriamiento: Cauchy
# tk = t0/(1+k)

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

# Temperatura inicial
t0 = mu * greedy_sol_cost / (- np.log(fi))

# Temperatura final
tf = t0 / (1 + k_max)



def simulated_an(t0,L,tf,slots,evaluation):

    t = t0

    current_solution      = generate_random_solution()
    cost_current_solution = evaluation.evaluate(current_solution)

    neighbors = Neighborhood(
        current_solution = current_solution,
        slots            = slots
    )

    k = 0

    while t >= tf:
        for cont in range(L):

            candidate_solution      = neighbors.get_neighbor_sa()
            cost_candidate_solution = evaluation.evaluate(candidate_solution)

            cost_difference = cost_candidate_solution - cost_current_solution

            if cost_difference < 0 or \
               np.random.rand() < np.exp( (-cost_difference) / t ):

                current_solution      = candidate_solution
                cost_current_solution = cost_candidate_solution 

                neighbors = Neighborhood(
                    current_solution = current_solution,
                    slots            = slots
                )
            
        t = t0 / (1 + k)
        k += 1

    return current_solution, cost_current_solution
                

In [ ]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    sa_solution, cost_sa = simulated_an(t0,L,tf,slots,evaluation)
    print(sa_solution)
    print(cost_sa)



##  TABU SEARCH

dudas sobre busqueda tabu:

1. Se implementa el criterio de aspiracion?
2. Se genera el vecino antes de comprobar y luego se descarta?
3. Reinicializamos cuando segun umbral/proporcion de aceptacion o segun iteraciones?

In [18]:
# externo
reboots = 4
total_iterations = 80

total_neighbors = 40
tenure = 4

slots = 5 # esto por que????


In [13]:
def update_frecuency(matrix, vector):

    for i in range(len(vector)):
        matrix[ vector[i] , i ] += 1

    return matrix


In [ ]:
np.seed(seeds[0])


current_solution      = generate_random_solution()
current_solution_cost = evaluation.evaluate(current_solution)

global_solution      = current_solution
global_solution_cost = current_solution_cost


# Matriz de frecuencias: inicializacion
frequency = np.ones((MAX_CAPACITY+1,TOTAL_STATIONS))

# Matriz frecuencias: actualizacion
frequency = update_frecuency(frequency, current_solution)

# tabu list
tl = TabuList(tenure)




for reboot in range(reboots):

    for iteration in range(int(total_iterations/reboots)):

        # genero vecinos -> 40 vecinos

        neighbors = Neighborhood(current_solution, slots)
        candidates = \
            [ neighbors.get_neighbor_ts() for n in range(total_neighbors) ]

        # ( solucion, movimiento, coste, esta_en_tabu_list )
        candidates_cost = [ 
            c[0], c[1], evaluation.evaluate(c[0]), c[1] in tl.tabu_list 
            for c in candidates 
        ]

        candidates_cost.sort(key= lambda x:x[2])

        # se cumple criterio de aspiracion -> mejorar la global siendo tabu
        if candidates_cost[0][2] < global_solution_cost:

            # aceptamos solucion
            current_solution      = candidates_cost[0][0]
            current_solution_cost = candidates_cost[0][2]

            # actualizamos global
            global_solution      = current_solution
            global_solution_cost = current_solution_cost

            # actualizamos tabu
            tl.add_movement(candidates_cost[0][1])

        else:

            # eliminamos los movimientos tabu-activos
            candidates_clean = [c for c in candidates_cost if not c[3]]

            # aceptamos solucion
            current_solution      = candidates_clean[0][0]
            current_solution_cost = candidates_clean[0][2]

            # actualizamos tabu
            tl.add_movement(candidates_clean[0][1])


        # actualizamos matriz de frecuencias
        frequency = update_frecuency(frequency, current_solution) 

    
    
            

        
        
        
         

    # lanzamos dado
    # estrategias:
        # aleatoria [0, 0.25)
        # matriz frecuencias + greedy : [0.25,0.75)
        # a partir de la mejor global: [0.75, 1)
    




In [9]:
prueba = [(1,2), (2,3), (1,2), (4,5)]

In [11]:
(1,2) in prueba

True

# ALGORITHMS ADJUSTMENT

# ANOTACIONES DIA 21/03/2022

Dia 4: defensa de la práctica
la bsuqueda  tabu deberá tardar más o menos lo que tarda el enfriamiento simulado.
No tenemos en cuenta si los vecinos se repiten o no (no lo combprobamos.)
velocidad de movimiento: 5 (nos dice cuantas posibilidades hay)
matriz de frecuencias: como se hace??? 
greedy: queremos hacer algo inversamente prorporcional a lo que ha salido (hacemos la inversa de cada casilla de la matriz de frecuencias???) y luego normalizo (?)
ruleta de probabilidad inversa -> buscar libreria, que seguro que está hecho en algun lado. Si no, está el pseudo en la práctica.

la condicion de parada: que sea mayor o igual al numero de iteraciones 
generar vecinos tiene la restriccion tabu
cogemos el algoritmo de busqueda local y vmos añadiendo trocitos
la parte que suele generar más dudas suele ser la parte del greedy


granular: tamaño de movimiento de huecos entre estaciones